
The Traffic Light Controlled Intersection problem on LeetCode requires you to simulate the operation of traffic lights at an intersection where multiple cars can pass based on certain constraints.

Problem Description
You are given three functions for cars coming from three different directions (East, North, and West), and these cars must pass through an intersection that is controlled by a traffic light. The traffic light allows only one direction to pass at a time.

Each function (carArrives) represents the arrival of a car at the intersection. If the traffic light is green for the direction from which the car is coming, the car can pass; otherwise, the car must wait until the light turns green.

Your task is to implement the simulation of this intersection, making sure cars pass in the correct order based on the traffic light rules.

Approach
The key here is to use synchronization to make sure only one direction's cars can pass at a time, as well as ensuring that cars are only allowed to pass when their traffic light is green.

A mutex lock or other thread synchronization mechanisms can be used to coordinate the passing of cars from different directions.

Steps:
Lock Mechanism: Use a lock to ensure only one car passes at a time.
Green Light Condition: Maintain a "green light" state that indicates which direction is allowed to pass.
Waiting: If a car arrives when the light is not green for its direction, it must wait.
Change of Light: After a car passes, change the light for the next set of cars.

In [2]:
import threading
from time import sleep

class TrafficLight:
    def __init__(self):
        # Initially, green light is for road 1 (East)
        self.green_light = 1
        self.lock = threading.Lock()

    def carArrived(self,
                   carId: int,           # ID of the car
                   roadId: int,          # ID of the road the car travels on. 1: East, 2: North, 3: West
                   direction: str,       # 'E' for east, 'N' for north, 'W' for west
                   turnGreen: 'Callable[[], None]',  # Use this function to turn light to green
                   crossCar: 'Callable[[], None]'):  # Use this function to make the car cross the intersection
        # Synchronize the access to the intersection
        with self.lock:
            # If the light is not green for the current road, turn it green
            if self.green_light != roadId:
                print(f"Traffic light turns green on road {roadId} for car {carId} from {direction}")
                turnGreen()
                self.green_light = roadId
            # Let the car pass through the intersection
            print(f"Car {carId} from {direction} is crossing the intersection")
            crossCar()

# Simulated car passing function
def crossCar():
    sleep(0.5)  # Simulate car crossing
    print("Car has crossed the intersection")

# Simulated light switching function
def turnGreen():
    sleep(0.1)  # Simulate time to switch the light

# Instantiate TrafficLight object
traffic_light = TrafficLight()

# Simulate cars arriving from different directions
cars = [
    (1, 1, 'East'),
    (2, 2, 'North'),
    (3, 1, 'East'),
    (4, 3, 'West'),
    (5, 2, 'North')
]

# Simulating each car arriving in separate threads
threads = []
for car in cars:
    carId, roadId, direction = car
    thread = threading.Thread(target=traffic_light.carArrived, args=(carId, roadId, direction, turnGreen, crossCar))
    threads.append(thread)
    thread.start()

# Wait for all threads to complete
for thread in threads:
    thread.join()


Car 1 from East is crossing the intersection
Car has crossed the intersection
Traffic light turns green on road 2 for car 2 from North
Car 2 from North is crossing the intersection
Car has crossed the intersection
Traffic light turns green on road 1 for car 3 from East
Car 3 from East is crossing the intersection
Car has crossed the intersection
Traffic light turns green on road 3 for car 4 from West
Car 4 from West is crossing the intersection
Car has crossed the intersection
Traffic light turns green on road 2 for car 5 from North
Car 5 from North is crossing the intersection
Car has crossed the intersection


Explanation:
Initialization: Each philosopher has access to two forks (left and right), and the self.forks array holds a lock for each fork.

Even and Odd Numbered Philosophers:

Philosophers with even indices will pick up the left fork first and then the right fork.
Philosophers with odd indices will pick up the right fork first and then the left fork.
This alternating pattern helps to avoid deadlock because it prevents a circular wait where every philosopher waits for another to release a fork.
Locks (with self.forks[i]): The with statement ensures that each philosopher can only pick up one fork at a time. After eating, they release the forks.

Time Complexity:
O(1): Since each philosopher just needs to acquire two locks (forks), eat, and release the locks.
Space Complexity:
O(1): We are using a fixed number of locks (one for each fork) and no extra space that scales with input size.

4

In [3]:
import threading
from time import sleep

class DiningPhilosophers:
    def __init__(self):
        # Each chopstick is a lock
        self.forks = [threading.Lock() for _ in range(5)]

    def wantsToEat(self,
                   philosopher: int,
                   pickLeftFork: 'Callable[[], None]',
                   pickRightFork: 'Callable[[], None]',
                   eat: 'Callable[[], None]',
                   putLeftFork: 'Callable[[], None]',
                   putRightFork: 'Callable[[], None]') -> None:
        left_fork = philosopher
        right_fork = (philosopher + 1) % 5

        if philosopher % 2 == 0:
            # Even-numbered philosophers pick up the left fork first
            with self.forks[left_fork]:
                with self.forks[right_fork]:
                    pickLeftFork()
                    pickRightFork()
                    eat()
                    putLeftFork()
                    putRightFork()
        else:
            # Odd-numbered philosophers pick up the right fork first
            with self.forks[right_fork]:
                with self.forks[left_fork]:
                    pickRightFork()
                    pickLeftFork()
                    eat()
                    putRightFork()
                    putLeftFork()

def pickLeftFork():
    print("Picked up left fork")

def pickRightFork():
    print("Picked up right fork")

def eat():
    print("Eating")
    sleep(1)  # Simulating the eating process

def putLeftFork():
    print("Put down left fork")

def putRightFork():
    print("Put down right fork")

# Instantiate the dining philosophers problem
dining_philosophers = DiningPhilosophers()

# Simulate philosophers trying to eat
def philosopher_dine(philosopher_id):
    dining_philosophers.wantsToEat(philosopher_id, pickLeftFork, pickRightFork, eat, putLeftFork, putRightFork)

# Create threads for each philosopher
threads = []
for i in range(5):
    thread = threading.Thread(target=philosopher_dine, args=(i,))
    threads.append(thread)
    thread.start()

# Wait for all threads to finish
for thread in threads:
    thread.join()


Picked up left fork
Picked up right fork
Eating
Picked up right fork
Picked up left fork
Eating
Put down right forkPut down left fork
Put down right fork
Picked up right fork
Picked up left fork
Eating

Put down left fork
Picked up left fork
Picked up right fork
Eating
Put down left fork
Put down right fork
Put down right fork
Put down left fork
Picked up left fork
Picked up right fork
Eating
Put down left fork
Put down right fork


# Print FooBar Alternately
 
Suppose you are given the following code:

class FooBar {
  public void foo() {
    for (int i = 0; i < n; i++) {
      print("foo");
    }
  }

  public void bar() {
    for (int i = 0; i < n; i++) {
      print("bar");
    }
  }
}

The same instance of FooBar will be passed to two different threads:

thread A will call foo(), while
thread B will call bar().
Modify the given program to output "foobar" n times.

 

Example 1:

Input: n = 1
Output: "foobar"
Explanation: There are two threads being fired asynchronously. One of them calls foo(), while the other calls bar().
"foobar" is being output 1 time.
Example 2:

Input: n = 2
Output: "foobarfoobar"
Explanation: "foobar" is being output 2 times.
 

To solve the problem where two threads alternately print "foo" and "bar" n times, we can use threading in Python and synchronization mechanisms like Locks or Condition variables to ensure that one thread prints "foo" before the other prints "bar".
 

Explanation:
    
Initialization:

- We define two locks: foo_lock and bar_lock.
- foo_lock is unlocked at the beginning because we want the foo method to execute first.
- bar_lock is initially locked to prevent bar from printing before foo.

foo method:

- The foo method prints "foo" when its lock (foo_lock) is acquired.
- After printing, it releases the bar_lock, allowing the bar method to proceed.

bar method:

- The bar method waits for the bar_lock to be released by foo before printing "bar".
- After printing, it releases the foo_lock, allowing the foo method to run again.

In [1]:
import threading

class FooBar:
    def __init__(self, n):
        self.n = n
        self.foo_lock = threading.Lock()
        self.bar_lock = threading.Lock()
        # Initially, bar_lock is locked to ensure "foo" starts first
        self.bar_lock.acquire()

    def foo(self, printFoo):
        for i in range(self.n):
            # Acquire the foo_lock to print "foo"
            self.foo_lock.acquire()
            # printFoo() outputs "foo". Do not change or remove this line.
            printFoo()
            # Release the bar_lock to let bar print "bar"
            self.bar_lock.release()

    def bar(self, printBar):
        for i in range(self.n):
            # Acquire the bar_lock to print "bar"
            self.bar_lock.acquire()
            # printBar() outputs "bar". Do not change or remove this line.
            printBar()
            # Release the foo_lock to let foo print "foo"
            self.foo_lock.release()
            
            
def printFoo():
    print("foo", end='')

def printBar():
    print("bar", end='')

foobar = FooBar(2)

thread_foo = threading.Thread(target=foobar.foo, args=(printFoo,))
thread_bar = threading.Thread(target=foobar.bar, args=(printBar,))

thread_foo.start()
thread_bar.start()

thread_foo.join()
thread_bar.join()

foobarfoobar

# Print Zero Even Odd
 
You have a function printNumber that can be called with an integer parameter and prints it to the console.

For example, calling printNumber(7) prints 7 to the console.

You are given an instance of the class ZeroEvenOdd that has three functions: zero, even, and odd. The same instance of ZeroEvenOdd will be passed to three different threads:

- Thread A: calls zero() that should only output 0's.
- Thread B: calls even() that should only output even numbers.
- Thread C: calls odd() that should only output odd numbers.
    
Modify the given class to output the series "010203040506..." where the length of the series must be 2n.

Implement the ZeroEvenOdd class:

ZeroEvenOdd(int n) Initializes the object with the number n that represents the numbers that should be printed.

void zero(printNumber) Calls printNumber to output one zero.

void even(printNumber) Calls printNumber to output one even number.

void odd(printNumber) Calls printNumber to output one odd number.


Explanation:

Initialization:
- n: This stores the total number of times we want the series to repeat.
- current: Keeps track of the current number being printed (used for even and odd).
- condition: A Condition object from the threading module, which is used to synchronize threads and manage when each thread should run.
- zero_turn and odd_turn: These flags control whether it is time to print 0 or an odd/even number.


zero method:
- This method is executed by Thread A and prints the number 0 for each iteration.
- It waits for its turn using condition.wait() and then sets zero_turn to False after printing 0, signaling either the even() or odd() methods to proceed.

even method:
- This method is executed by Thread B and prints even numbers starting from 2.
- It waits until zero_turn is False and it's not the odd turn (odd_turn is True). After printing the even number, it signals the zero thread to run next.

odd method:
- This method is executed by Thread C and prints odd numbers starting from 1.
- It waits for the same condition as even(), but prints the odd numbers when odd_turn is True. After printing, it signals the zero thread.

In [7]:
import threading

class ZeroEvenOdd:
    def __init__(self, n):
        self.n = n
        self.current = 1
        self.condition = threading.Condition()
        self.zero_turn = True
        self.odd_turn = True

    def zero(self, printNumber):
        for i in range(self.n):
            with self.condition:
                while not self.zero_turn:
                    self.condition.wait()
                # printNumber(0) outputs "0". Do not change or remove this line.
                printNumber(0)
                self.zero_turn = False
                self.condition.notify_all()

    def even(self, printNumber):
        for i in range(2, self.n + 1, 2):
            with self.condition:
                while self.zero_turn or self.odd_turn:
                    self.condition.wait()
                # printNumber(i) outputs the even number. Do not change or remove this line.
                printNumber(i)
                self.zero_turn = True
                self.odd_turn = True
                self.condition.notify_all()

    def odd(self, printNumber):
        for i in range(1, self.n + 1, 2):
            with self.condition:
                while self.zero_turn or not self.odd_turn:
                    self.condition.wait()
                # printNumber(i) outputs the odd number. Do not change or remove this line.
                printNumber(i)
                self.zero_turn = True
                self.odd_turn = False
                self.condition.notify_all()

                
def printNumber(x):
    print(x, end='')

n = 5
zeroEvenOdd = ZeroEvenOdd(n)

thread_zero = threading.Thread(target=zeroEvenOdd.zero, args=(printNumber,))
thread_even = threading.Thread(target=zeroEvenOdd.even, args=(printNumber,))
thread_odd = threading.Thread(target=zeroEvenOdd.odd, args=(printNumber,))

thread_zero.start()
thread_even.start()
thread_odd.start()

thread_zero.join()
thread_even.join()
thread_odd.join()


0102030405